In [51]:
import collections
import nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier, MaxentClassifier, SklearnClassifier
import csv
from sklearn import cross_validation
from sklearn.svm import LinearSVC, SVC
import random
from nltk.corpus import stopwords
import itertools
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import *

In [52]:
from __future__ import division #To avoid integer division
from operator import itemgetter
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [53]:
data = pd.read_csv("data_set.csv", low_memory=False)
data = data[['reviews.rating', 'reviews.text']]
train, test = train_test_split(data, test_size=0.2)

In [54]:
pos_rev = []
# pos_rev_date = []
neg_rev = []
# neg_rev_date = []
neut_rev = []
# neut_rev_date = []

train['reviews.rating'] = pd.to_numeric(train['reviews.rating'], errors='coerce')
train = train.dropna(subset=['reviews.rating'])
train['reviews.rating'] = train['reviews.rating'].astype(int)

# a = 0
for col, row in train.iterrows():
    if int(row["reviews.rating"]) > 3:
        pos_rev.append(row["reviews.text"])
#         pos_rev_date.append(row["reviews.date"])
    elif int(row["reviews.rating"]) < 3:
        neg_rev.append(row["reviews.text"])
#         neg_rev_date.append(row["reviews.date"])
    else:
        neut_rev.append(row["reviews.text"])
#         neut_rev_date.append(row["reviews.date"])

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [55]:
def word_split(data):    
    data_new = []
    for word in data:
        try:
            word_filter = [i.lower() for i in word.split()]
            data_new.append(word_filter)
        except:
            print (' ')
    return data_new
 
def word_split_sentiment(data):
    data_new = []
    for (word, sentiment) in data:
        try:
            word_filter = [i.lower() for i in word.split()]
            data_new.append((word_filter, sentiment))
        except:
            print (' ')
    return data_new
    
def word_feats(words):    
    return dict([(word, True) for word in words])

In [ ]:
from nltk.metrics import *

def evaluate_classifier(featx):
    
    negfeats = [(featx(f), 'neg') for f in word_split(neg_rev)]
    posfeats = [(featx(f), 'pos') for f in word_split(pos_rev)]
    neutfeats = [(featx(f), 'neut') for f in word_split(neut_rev)]
    
#     print (posfeats)
        
    negcutoff = int(len(negfeats)*3/4)
    poscutoff = int(len(posfeats)*3/4)
    neutcutoff = int(len(neutfeats)*3/4)
    
#     print (len(posfeats))
#     print (len(negfeats))
#     print (len(neutfeats))
    
    trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff] + neutfeats[:neutcutoff]
    testfeats = negfeats[negcutoff:] + posfeats[poscutoff:] + neutfeats[neutcutoff:]
    
    # using 3 classifiers
    classifier_list = ['nb', 'maxent', 'svm']
        
    for cl in classifier_list:
        if cl == 'maxent':
            classifierName = 'Maximum Entropy'
            classifier = MaxentClassifier.train(trainfeats, 'GIS', trace=0, encoding=None, labels=None, gaussian_prior_sigma=0, max_iter = 1)
        elif cl == 'svm':
            classifierName = 'SVM'
            classifier = SklearnClassifier(LinearSVC(), sparse=False)
            classifier.train(trainfeats)
        else:
            classifierName = 'Naive Bayes'
            classifier = NaiveBayesClassifier.train(trainfeats)
            
        refsets = collections.defaultdict(set)
        testsets = collections.defaultdict(set)
 
        for i, (feats, label) in enumerate(testfeats):
                refsets[label].add(i)
                observed = classifier.classify(feats)
                testsets[observed].add(i)

        accuracy = nltk.classify.util.accuracy(classifier, testfeats)
        pos_precision = precision(refsets['pos'], testsets['pos'])
        pos_recall = recall(refsets['pos'], testsets['pos'])
        pos_fmeasure = f_measure(refsets['pos'], testsets['pos'])
        neg_precision = precision(refsets['neg'], testsets['neg'])
        neg_recall = recall(refsets['neg'], testsets['neg'])
        neg_fmeasure = f_measure(refsets['neg'], testsets['neg'])
        neut_precision = precision(refsets['neut'], testsets['neut'])
        neut_recall = recall(refsets['neut'], testsets['neut'])
        neut_fmeasure = f_measure(refsets['neut'], testsets['neut'])
        
        if pos_precision is None:
            pos_precision = 0
        if pos_recall is None:
            pos_recall = 0
        if pos_fmeasure is None:
            pos_fmeasure = 0
        if neg_precision is None:
            neg_precision = 0
        if neg_recall is None:
            neg_recall = 0
        if neg_fmeasure is None:
            neg_fmeasure = 0
        if neut_precision is None:
            neut_precision = 0
        if neut_recall is None:
            neut_recall = 0
        if neut_fmeasure is None:
            neut_fmeasure = 0
        
        print (' ')
        print ('---------------------------------------')
        print ('SINGLE FOLD RESULT ' + '(' + classifierName + ')')
        print ('---------------------------------------')
        print ('accuracy:', accuracy)
        print ('precision', (pos_precision + neg_precision + neut_precision) / 3)
        print ('recall', (pos_recall + neg_recall + neut_recall) / 3)
        print ('f-measure', (pos_fmeasure + neg_fmeasure + neut_fmeasure) / 3)
                
        #classifier.show_most_informative_features()
        
    trainfeats = negfeats + posfeats + neutfeats
    
    # SHUFFLE TRAIN SET
    # As in cross validation, the test chunk might have only negative or only positive data    
    random.shuffle(trainfeats)    
    n = 5 # 5-fold cross-validation    
    
    for cl in classifier_list:
        
        subset_size = int(len(trainfeats) / n)
        accuracy = []
        pos_precision = []
        pos_recall = []
        neg_precision = []
        neg_recall = []
        pos_fmeasure = []
        neg_fmeasure = []
        neut_precision = []
        neut_recall = []
        neut_fmeasure = []
        cv_count = 1
        for i in range(n):        
            testing_this_round = trainfeats[i*subset_size:][:subset_size]
            training_this_round = trainfeats[:i*subset_size] + trainfeats[(i+1)*subset_size:]
            
            if cl == 'maxent':
                classifierName = 'Maximum Entropy'
                classifier = MaxentClassifier.train(training_this_round, 'GIS', trace=0, encoding=None, labels=None, gaussian_prior_sigma=0, max_iter = 1)
            elif cl == 'svm':
                classifierName = 'SVM'
                classifier = SklearnClassifier(LinearSVC(), sparse=False)
                classifier.train(training_this_round)
            else:
                classifierName = 'Naive Bayes'
                classifier = NaiveBayesClassifier.train(training_this_round)
                    
            refsets = collections.defaultdict(set)
            testsets = collections.defaultdict(set)
            for i, (feats, label) in enumerate(testing_this_round):
                refsets[label].add(i)
                observed = classifier.classify(feats)
                testsets[observed].add(i)
            
            cv_accuracy = 0
            cv_pos_precision = 0
            cv_pos_recall = 0
            cv_pos_fmeasure = 0
            cv_neg_precision = 0
            cv_neg_recall = 0
            cv_neg_fmeasure = 0
            cv_neut_precision = 0
            cv_neut_recall = 0
            cv_neut_fmeasure = 0
            
            cv_accuracy = nltk.classify.util.accuracy(classifier, testing_this_round)
            cv_pos_precision = precision(refsets['pos'], testsets['pos'])
            cv_pos_recall = recall(refsets['pos'], testsets['pos'])
            cv_pos_fmeasure = f_measure(refsets['pos'], testsets['pos'])
            cv_neg_precision = precision(refsets['neg'], testsets['neg'])
            cv_neg_recall = recall(refsets['neg'], testsets['neg'])
            cv_neg_fmeasure = f_measure(refsets['neg'], testsets['neg'])
            cv_neut_precision = precision(refsets['neut'], testsets['neut'])
            cv_neut_recall = recall(refsets['neut'], testsets['neut'])
            cv_neut_fmeasure = f_measure(refsets['neut'], testsets['neut'])
            
            if cv_pos_precision is None:
                cv_pos_precision = 0
            if cv_pos_recall is None:
                cv_pos_recall = 0
            if cv_pos_fmeasure is None:
                cv_pos_fmeasure = 0
            if cv_neg_precision is None:
                cv_neg_precision = 0
            if cv_neg_recall is None:
                cv_neg_recall = 0
            if cv_neg_fmeasure is None:
                cv_neg_fmeasure = 0
            if cv_neut_precision is None:
                cv_neut_precision = 0
            if cv_neut_recall is None:
                cv_neut_recall = 0
            if cv_neut_fmeasure is None:
                cv_neut_fmeasure = 0

                    
            accuracy.append(cv_accuracy)
            pos_precision.append(cv_pos_precision)
            pos_recall.append(cv_pos_recall)
            neg_precision.append(cv_neg_precision)
            neg_recall.append(cv_neg_recall)
            pos_fmeasure.append(cv_pos_fmeasure)
            neg_fmeasure.append(cv_neg_fmeasure)
            neut_precision.append(cv_neut_precision)
            neut_recall.append(cv_neut_recall)
            neut_fmeasure.append(cv_neut_fmeasure)
            
            cv_count += 1
        
                
        print ('---------------------------------------')
        print ('N-FOLD CROSS VALIDATION RESULT ' + '(' + classifierName + ')')
        print ('---------------------------------------')
        print ('accuracy:', sum(accuracy) / n)
        print ('precision', (sum(pos_precision)/n + sum(neg_precision)/n + sum(neut_precision)/n) / 3)
        print ('recall', (sum(pos_recall)/n + sum(neg_recall)/n + sum(neut_recall)/n) / 3)
        print ('f-measure', (sum(pos_fmeasure)/n + sum(neg_fmeasure)/n + sum(neut_fmeasure)/n) / 3)
        print ('')
        

In [ ]:
evaluate_classifier(word_feats)